# WGCNA (Weighted Correlation Network Analysis) 的数学原理

## 目标：基于基因表达矩阵，识别出具有相似表达模式的一堆基因（模块），并将其与外部表型数据（如疾病、治疗和时间）相关联。

## 1. 输入基因表达矩阵

设有基因表达矩阵：

$$
X \in \mathbb{R}^{n \times p}
$$

- $n$：样本数
- $p$：基因数
- 第 $i$ 行 $x_i = (x_{i1}, x_{i2}, \ldots, x_{ip})$ 是第 $i$ 个样本的表达向量

---

## 2. 相似性矩阵（Similarity Matrix）

定义任意两个基因 $i, j$ 的皮尔逊相关系数：

$$
s_{ij} = \text{cor}(x_{\cdot i}, x_{\cdot j}) = \frac{\sum_{k=1}^n (x_{ki} - \bar{x}_{\cdot i})(x_{kj} - \bar{x}_{\cdot j})}{\sqrt{\sum_{k=1}^n (x_{ki} - \bar{x}_{\cdot i})^2} \sqrt{\sum_{k=1}^n (x_{kj} - \bar{x}_{\cdot j})^2}}
$$

其中 $\bar{x}_{\cdot i}$ 是第 $i$ 个基因的样本均值。

- 得到相似性矩阵 $S = (s_{ij})$，对称，且 $s_{ii} = 1$。

---

## 3. 邻接矩阵（Adjacency Matrix）

定义邻接矩阵 $A = (a_{ij})$：

$$
a_{ij} = |s_{ij}|^\beta
$$

- $\beta > 1$ 是 soft-thresholding power
- 通常选择能让网络呈现 scale-free topology 的 $\beta$


### Scale-free Topology 


在一个无向图 $G = (V, E)$ 中：

- $V$ 是节点集合，$|V| = N$
- $E$ 是边集合
- 每个节点 $v \in V$ 有一个度（degree）$k_v$，表示连接的边的数量。

定义节点度的概率分布函数 $P(k)$ 为：

$$
P(k) = \frac{\text{节点度数为 } k \text{ 的节点数量}}{N}
$$

一个网络具有 **scale-free topology**，当且仅当其度分布满足以下幂律关系：

$$
P(k) \propto k^{-\gamma}, \quad k \geq k_{\text{min}}
$$

其中：
- $\gamma > 1$ 是幂律指数，通常在 $(2,3)$ 区间。
- $k_{\text{min}}$ 是度分布开始遵循幂律行为的最小度值。

换句话说，存在正常数 $C > 0$，使得对于所有 $k \geq k_{\text{min}}$：

$$
P(k) = C k^{-\gamma}
$$


#### 对数变换后的线性关系

取对数两边，得到：

$$
\log P(k) = -\gamma \log k + \log C
$$

即在 $\log$-$\log$ 坐标下，$P(k)$ 关于 $k$ 呈现一条斜率为 $-\gamma$ 的直线。



#### 节点度的幂律分布特性

幂律分布具有以下数学性质：

1. **无特征尺度（scale-free）**  
   度分布中不存在一个典型的平均度，度可以在多个数量级范围内变化。  
   对于幂律分布，期望值和方差可能发散，具体取决于 $\gamma$：

   - 当 $2 < \gamma \leq 3$，期望有限，但方差无限。
   - 当 $1 < \gamma \leq 2$，期望和方差都发散。

2. **长尾特性（heavy-tailed）**  
   大度节点（hub nodes）虽然稀少，但出现概率远高于指数衰减或正态分布。



#### 度分布的归一化常数

为了使 $P(k)$ 是一个有效的概率分布，需要归一化，即：

$$
\sum_{k = k_{\text{min}}}^{\infty} P(k) = 1
$$

代入幂律表达式：

$$
C \sum_{k = k_{\text{min}}}^{\infty} k^{-\gamma} = 1
$$

因此归一化常数 $C$ 由：

$$
C = \left( \sum_{k = k_{\text{min}}}^{\infty} k^{-\gamma} \right)^{-1}
$$

确定。

对于连续近似（$k$ 取连续值而非离散整数），归一化可以通过积分近似：

$$
\int_{k_{\text{min}}}^{\infty} C k^{-\gamma} \, dk = 1
$$

解得：

$$
C = (\gamma - 1) k_{\text{min}}^{\gamma - 1}
$$


#### Scale-free 网络的总结定义

**数学上定义**：

一个图 $G = (V, E)$ 满足：

- 存在 $\gamma > 1$，使得度分布满足 $P(k) \sim k^{-\gamma}$；
- 其中 $k \geq k_{\text{min}}$；
- 并且 $\gamma$ 通常落在 $[2,3]$ 范围。

则称 $G$ 是一个 **scale-free network**。


---

## 4. 拓扑重叠矩阵（Topological Overlap Matrix, TOM）

定义共同邻居数：

$$
l_{ij} = \sum_{u=1}^{p} a_{iu} a_{ju}
$$

- $l_{ij}$ 表示节点 $i$ 和 $j$ 的共同邻居连接强度之和。


节点度（Degree）：

$$
k_i = \sum_{u=1}^{p} a_{iu}
$$

- $k_i$ 是节点 $i$ 连接到其他所有节点的总权重。


### 拓扑重叠矩阵元素：

$$
\text{TOM}_{ij} = \frac{l_{ij} + a_{ij}}{\min(k_i, k_j) + 1 - a_{ij}}
$$

- 分子：$l_{ij} + a_{ij}$，即共同邻居连接强度加上直接连接强度。
- 分母：$\min(k_i, k_j) + 1 - a_{ij}$，是一个归一化因子，防止大hub节点拉偏。


### 拓扑重叠矩阵的性质：

- $\text{TOM}_{ii} = 1$，每个节点与自己的相似性最大。
- $0 \leq \text{TOM}_{ij} \leq 1$，标准化到0到1之间。

### 解释总结：

- **TOM越接近1**，意味着节点 $i$ 和 $j$ **共享的邻居越多**，并且可能直接连接。
- **TOM越接近0**，意味着节点 $i$ 和 $j$ 几乎没有共同邻居，关系疏远。


### TOM 计算具体例子

#### 假设一个小型邻接矩阵 $A$：

设有4个节点（$p=4$），邻接矩阵如下（加权的）：

$$
A = \begin{pmatrix}
0 & 1 & 1 & 0 \\
1 & 0 & 1 & 1 \\
1 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 \\
\end{pmatrix}
$$

解释一下：
- 节点 1 和 2 有连边，权重 1
- 节点 1 和 3 有连边，权重 1
- 节点 2 和 3 有连边，权重 1
- 节点 2 和 4 有连边，权重 1

其他地方没有边（用0表示）。



#### 第一步：计算节点度 $k_i$

节点度是该节点所有连边权重的和：

- $k_1 = a_{12} + a_{13} = 1 + 1 = 2$
- $k_2 = a_{21} + a_{23} + a_{24} = 1 + 1 + 1 = 3$
- $k_3 = a_{31} + a_{32} = 1 + 1 = 2$
- $k_4 = a_{42} = 1$



#### 第二步：计算共同邻居数 $l_{ij}$

定义：

$$
l_{ij} = \sum_{u=1}^{p} a_{iu} a_{ju}
$$

就是看 $i$ 和 $j$ 是否有共同连着的节点。

具体计算：

- $l_{12} = (a_{11}a_{21}) + (a_{12}a_{22}) + (a_{13}a_{23}) + (a_{14}a_{24})$
  
  $= (0)(1) + (1)(0) + (1)(1) + (0)(1) = 1$

- $l_{13} = (a_{11}a_{31}) + (a_{12}a_{32}) + (a_{13}a_{33}) + (a_{14}a_{34})$

  $= (0)(1) + (1)(1) + (1)(0) + (0)(0) = 1$

- $l_{14} = (a_{11}a_{41}) + (a_{12}a_{42}) + (a_{13}a_{43}) + (a_{14}a_{44})$

  $= (0)(0) + (1)(1) + (1)(0) + (0)(0) = 1$

- $l_{23} = (a_{21}a_{31}) + (a_{22}a_{32}) + (a_{23}a_{33}) + (a_{24}a_{34})$

  $= (1)(1) + (0)(1) + (1)(0) + (1)(0) = 1$

- $l_{24} = (a_{21}a_{41}) + (a_{22}a_{42}) + (a_{23}a_{43}) + (a_{24}a_{44})$

  $= (1)(0) + (0)(1) + (1)(0) + (1)(0) = 0$

- $l_{34} = (a_{31}a_{41}) + (a_{32}a_{42}) + (a_{33}a_{43}) + (a_{34}a_{44})$

  $= (1)(0) + (1)(1) + (0)(0) + (0)(0) = 1$



#### 第三步：计算 $\text{TOM}_{ij}$

公式：

$$
\text{TOM}_{ij} = \frac{l_{ij} + a_{ij}}{\min(k_i, k_j) + 1 - a_{ij}}
$$

逐对算！



- **$\text{TOM}_{12}$**

  - $l_{12} = 1$, $a_{12} = 1$
  - $k_1 = 2$, $k_2 = 3$
  - $\min(k_1, k_2) = 2$

  $$ 
  \text{TOM}_{12} = \frac{1 + 1}{2 + 1 - 1} = \frac{2}{2} = 1
  $$



- **$\text{TOM}_{13}$**

  - $l_{13} = 1$, $a_{13} = 1$
  - $k_1 = 2$, $k_3 = 2$
  - $\min(k_1, k_3) = 2$

  $$
  \text{TOM}_{13} = \frac{1 + 1}{2 + 1 - 1} = \frac{2}{2} = 1
  $$



- **$\text{TOM}_{14}$**

  - $l_{14} = 1$, $a_{14} = 0$
  - $k_1 = 2$, $k_4 = 1$
  - $\min(k_1, k_4) = 1$

  $$
  \text{TOM}_{14} = \frac{1 + 0}{1 + 1 - 0} = \frac{1}{2} = 0.5
  $$



- **$\text{TOM}_{23}$**

  - $l_{23} = 1$, $a_{23} = 1$
  - $k_2 = 3$, $k_3 = 2$
  - $\min(k_2, k_3) = 2$

  $$
  \text{TOM}_{23} = \frac{1 + 1}{2 + 1 - 1} = \frac{2}{2} = 1
  $$



- **$\text{TOM}_{24}$**

  - $l_{24} = 0$, $a_{24} = 1$
  - $k_2 = 3$, $k_4 = 1$
  - $\min(k_2, k_4) = 1$

  $$
  \text{TOM}_{24} = \frac{0 + 1}{1 + 1 - 1} = \frac{1}{1} = 1
  $$



- **$\text{TOM}_{34}$**

  - $l_{34} = 1$, $a_{34} = 0$
  - $k_3 = 2$, $k_4 = 1$
  - $\min(k_3, k_4) = 1$

  $$
  \text{TOM}_{34} = \frac{1 + 0}{1 + 1 - 0} = \frac{1}{2} = 0.5
  $$


#### 最终 $\text{TOM}$ 矩阵是：

$$
\text{TOM} = \begin{pmatrix}
1 & 1 & 1 & 0.5 \\
1 & 1 & 1 & 1 \\
1 & 1 & 1 & 0.5 \\
0.5 & 1 & 0.5 & 1 \\
\end{pmatrix}
$$


---

## 5. 基因模块识别（Gene Module Detection）

定义距离矩阵：

$$
d_{ij} = 1 - \text{TOM}_{ij}
$$

- 使用分层聚类（hierarchical clustering）
- 采用 Dynamic Tree Cut 自动剪切聚类树
- 每个模块 $M_k$ 是一组基因集合


### 5.1 定义拓扑重叠矩阵（TOM）

给定邻接矩阵 $A = (a_{ij})$，节点度为：

$$
k_i = \sum_{u=1}^p a_{iu}
$$

共同邻居数为：

$$
l_{ij} = \sum_{u=1}^p a_{iu} a_{ju}
$$

拓扑重叠矩阵元素定义为：

$$
\text{TOM}_{ij} = \frac{l_{ij} + a_{ij}}{\min(k_i, k_j) + 1 - a_{ij}}
$$

性质：
- $\text{TOM}_{ii} = 1$
- $0 \leq \text{TOM}_{ij} \leq 1$



### 5.2 定义距离矩阵 $D$

基于拓扑重叠矩阵，定义距离矩阵 $D = (d_{ij})$：

$$
d_{ij} = 1 - \text{TOM}_{ij}
$$

性质：
- $d_{ii} = 0$
- $0 \leq d_{ij} \leq 1$



### 5.3 分层聚类（Hierarchical Clustering）

在距离矩阵 $D$ 上进行分层聚类，构建树状结构（dendrogram）$\mathcal{T}$：

- 初始状态：每个基因为独立类。
- 合并策略：根据 linkage function $\mathcal{L}(C_1, C_2)$ 最小化类间距离。
- 通常采用平均距离（average linkage）：

$$
\mathcal{L}(C_1, C_2) = \frac{1}{|C_1||C_2|} \sum_{i \in C_1} \sum_{j \in C_2} d_{ij}
$$



### 5.4 动态剪切树（Dynamic Tree Cut）

在聚类树 $\mathcal{T}$ 上应用动态剪切算法，识别模块：

- 定义子树 $M_k \subseteq V$，满足：
  - 子树内部相似性（TOM高，$d$低）
  - 子树之间异质性（TOM低，$d$高）
- 动态剪切自动确定模块数量和模块大小。

最终得到一组模块集合：

$$
\{ M_1, M_2, \ldots, M_K \}
$$

其中 $M_k \subseteq V$，且：

$$
\bigcup_{k=1}^{K} M_k = V
$$

（所有基因被完全分配进至少一个模块，不重不漏）


### 数学流程总结

1. 输入：邻接矩阵 $A$。
2. 计算拓扑重叠矩阵 $\text{TOM}$。
3. 计算距离矩阵 $d_{ij} = 1 - \text{TOM}_{ij}$。
4. 基于 $D$，构建分层聚类树 $\mathcal{T}$。
5. 应用动态剪切，得到模块划分 $\{ M_k \}$。



### ✨ 额外数学性质

- 聚类过程中，树 $\mathcal{T}$ 保证距离的超度量性质（ultrametric property）：
  
  对任意三个节点 $i, j, k$，有：

  $$
  d_{ij} \leq \max(d_{ik}, d_{jk})
  $$

- 动态剪切允许模块大小不均匀，避免固定半径剪切的局限性。



### 第一部分：聚类树 $\mathcal{T}$ 为什么满足超度量性质？

首先回顾超度量（Ultrametric）的**数学定义**：

对任意三个点 $i, j, k$，超度量要求：

$$
d_{ij} \leq \max(d_{ik}, d_{jk})
$$

比普通三角不等式还更严格（普通的是 $d_{ij} \leq d_{ik} + d_{kj}$）。


**而分层聚类（Hierarchical clustering）**的过程自然保证了超度量成立！

原因是：
- 分层聚类是**合并最近的两个簇**，每次合并实际上更新了内部所有节点的**新距离**。
- 新簇内部的所有节点之间的距离，都被定义成「通过合并步骤的高度」。

在**平均链接法（average linkage）**中，比如合并 $i$ 和 $j$，新的距离被定义为两个簇之间所有元素的平均。
在构造 dendrogram 的时候，任意两点之间的"路径长度"是从根到最近公共祖先的距离。

而且在树上，**任意两个节点的距离，只取决于它们的最近公共祖先（LCA, Lowest Common Ancestor）**的高度！

所以本质上，对于任意三点 $i, j, k$：

- $d_{ij}$ 是 $i$ 和 $j$ 的最近公共祖先的高度。
- $d_{ik}$ 是 $i$ 和 $k$ 的最近公共祖先的高度。
- $d_{jk}$ 是 $j$ 和 $k$ 的最近公共祖先的高度。

因为公共祖先只能更高不能更低，所以一定有：

$$
d_{ij} \leq \max(d_{ik}, d_{jk})
$$

**超度量自然成立。**

✅ **核心一句话总结**：

> **树上的距离是取最近公共祖先（LCA）的高度，天然满足 ultrametric！**



### 第二部分：Dynamic Tree Cut 为什么允许模块大小不均匀？

**传统剪树的方法**是什么？

- 固定一个剪切高度（比如 $h = 0.25$），然后一刀切掉所有高于 $h$ 的地方。
- 这样剪出来的模块大小非常均匀（每个模块相似大小），因为都是按照一个固定门槛划分的。

⚡ **但真实的基因网络世界可不是均匀的！**
- 有些模块很紧密（内部距离很小），可以早早切割。
- 有些模块很松散（内部距离大一点），要到更高的位置才合理切开。
- 不同模块天然密度不同！



**Dynamic Tree Cut 的数学核心：**

它根据**局部树结构自适应地决定剪切点**，而不是用统一的全局高度。

- 小模块：如果一个子树内部紧密，Dynamic Tree Cut 会在比较低的位置剪断。
- 大模块：如果一个子树内部疏松，Dynamic Tree Cut 允许它继续合并，直到达到内部一致性。

也就是说：
- Dynamic Tree Cut 允许不同模块的剪切门槛**是动态变化的**。
- 不强制模块大小统一。

---

## 6. 模块特征向量（Module Eigengene, ME）


### 6.1 基本设置

给定一个模块 $M_k$：

- 包含 $|M_k|$ 个基因。
- 对应的基因表达矩阵记为：

$$
X_k \in \mathbb{R}^{n \times |M_k|}
$$

其中：
- $n$ 是样本数（即每行是一个样本）
- $|M_k|$ 是模块中基因的数量（即每列是一个基因的表达）



### 6.2 定义模块特征向量（Module Eigengene）

模块特征向量 $\text{ME}_k \in \mathbb{R}^n$ 定义为 $X_k$ 的第一主成分（first principal component）。

即：

$$
\text{ME}_k = X_k v_1
$$

其中：

- $v_1 \in \mathbb{R}^{|M_k|}$ 是矩阵 $X_k$ 的协方差矩阵的最大特征值对应的特征向量。

更数学地说：

- 协方差矩阵：

$$
\Sigma_k = \frac{1}{n-1} X_k^\top X_k \in \mathbb{R}^{|M_k| \times |M_k|}
$$

- 求解特征值问题：

$$
\Sigma_k v_1 = \lambda_1 v_1
$$

其中：
- $\lambda_1$ 是最大的特征值（$\lambda_1 \geq \lambda_2 \geq \dots$）
- $v_1$ 是对应的单位特征向量（$\|v_1\| = 1$）

然后：

$$
\text{ME}_k = X_k v_1
$$



### 6.3 为什么这么定义？

- $v_1$ 给出了**模块内部基因表达变化**最主要的方向（最大方差方向）。
- 投影到 $v_1$ 后得到的 $\text{ME}_k$，是每个样本在这个方向上的坐标。
- $\text{ME}_k$ 代表了整个模块在各样本中的**主导表达趋势**。

简单说就是：

> **模块一堆基因太复杂？没关系，压缩成一个最能代表它们整体变化的向量，叫做 ME。**


### 优化问题理解

模块特征向量 $\text{ME}_k$ 也可以理解为：

$$
v_1 = \arg\max_{\|v\|=1} \operatorname{Var}(X_k v)
$$

即找到方差最大的投影方向。

### 📚 纯数学推导：模块特征向量（Module Eigengene）



#### 1. 问题设定

假设中心化后的模块基因表达矩阵为：

$$
X_k \in \mathbb{R}^{n \times p}
$$

例如：

$$
X_k = \begin{pmatrix}
2 & 3 & 5 \\
4 & 6 & 8 \\
1 & 1 & 2 \\
5 & 7 & 10 \\
3 & 4 & 6 \\
\end{pmatrix}
\in \mathbb{R}^{5 \times 3}
$$



#### 2. 投影与方差

投影到方向 \( v \in \mathbb{R}^p \)，得到投影向量：

$$
z = X_k v
$$

投影后的样本方差为：

$$
\operatorname{Var}(z) = \frac{1}{n - 1} z^\top z
$$

代入 \( z = X_k v \)，展开得：

$$
\operatorname{Var}(z) = \frac{1}{n - 1} v^\top X_k^\top X_k v
$$



#### 3. 协方差矩阵

定义协方差矩阵：

$$
\Sigma_k = \frac{1}{n - 1} X_k^\top X_k
$$

于是投影方差可以简化为：

$$
\operatorname{Var}(z) = v^\top \Sigma_k v
$$



#### 4. 最优化问题

最大投影方差方向 \( v_1 \) 可由下式求得：

$$
v_1 = \arg\max_{\|v\| = 1} v^\top \Sigma_k v
$$



#### 5. 解析解

根据线性代数理论，最优解 \( v_1 \) 为 \( \Sigma_k \) 的最大特征值对应的单位特征向量。



#### 6. 模块特征向量定义

模块特征向量（Module Eigengene）定义为：

$$
\text{ME}_k = X_k v_1
$$



### 模块特征向量（ME）数学例子


#### 假设一个小模块表达矩阵 $X_k$

假设模块 $M_k$ 里有3个基因，5个样本，表达矩阵是：

$$
X_k = \begin{pmatrix}
2 & 3 & 5 \\
4 & 6 & 8 \\
1 & 1 & 2 \\
5 & 7 & 10 \\
3 & 4 & 6 \\
\end{pmatrix}
\in \mathbb{R}^{5 \times 3}
$$

- 每行是一个样本
- 每列是一个基因



#### 第一步：中心化（mean centering）

PCA需要先把每列减去均值，使得每个基因的表达均值为0。

首先，计算每列的均值：

- 第1列（基因1）均值：

$$
\bar{x}_1 = \frac{2 + 4 + 1 + 5 + 3}{5} = 3
$$

- 第2列（基因2）均值：

$$
\bar{x}_2 = \frac{3 + 6 + 1 + 7 + 4}{5} = 4.2
$$

- 第3列（基因3）均值：

$$
\bar{x}_3 = \frac{5 + 8 + 2 + 10 + 6}{5} = 6.2
$$

进行中心化：

$$
\tilde{X}_k = X_k - \text{mean}(X_k)
$$

得到中心化后的矩阵：

$$
\tilde{X}_k = \begin{pmatrix}
-1 & -1.2 & -1.2 \\
1 & 1.8 & 1.8 \\
-2 & -3.2 & -4.2 \\
2 & 2.8 & 3.8 \\
0 & -0.2 & -0.2 \\
\end{pmatrix}
$$



#### 第二步：计算协方差矩阵

协方差矩阵定义为：

$$
\Sigma_k = \frac{1}{n-1} \tilde{X}_k^\top \tilde{X}_k
$$

$n = 5$，所以除以4。

先计算 $\tilde{X}_k^\top \tilde{X}_k$：

$$
\tilde{X}_k^\top \tilde{X}_k = \begin{pmatrix}
(-1)(-1) + (1)(1) + (-2)(-2) + (2)(2) + (0)(0) & \cdots & \cdots \\
\cdots & \cdots & \cdots \\
\cdots & \cdots & \cdots \\
\end{pmatrix}
$$

自己算一下每个元素：

- (1,1) 元素：

$$
(-1)^2 + (1)^2 + (-2)^2 + (2)^2 + (0)^2 = 1+1+4+4+0 = 10
$$

- (1,2) 元素：

$$
(-1)(-1.2) + (1)(1.8) + (-2)(-3.2) + (2)(2.8) + (0)(-0.2) = 1.2+1.8+6.4+5.6+0 = 15
$$

- (1,3) 元素：

$$
(-1)(-1.2) + (1)(1.8) + (-2)(-4.2) + (2)(3.8) + (0)(-0.2) = 1.2+1.8+8.4+7.6+0 = 19
$$

- (2,2) 元素：

$$
(-1.2)^2 + (1.8)^2 + (-3.2)^2 + (2.8)^2 + (-0.2)^2 = 1.44+3.24+10.24+7.84+0.04=22.8
$$

- (2,3) 元素：

$$
(-1.2)(-1.2) + (1.8)(1.8) + (-3.2)(-4.2) + (2.8)(3.8) + (-0.2)(-0.2) = 1.44+3.24+13.44+10.64+0.04=28.8
$$

- (3,3) 元素：

$$
(-1.2)^2 + (1.8)^2 + (-4.2)^2 + (3.8)^2 + (-0.2)^2 = 1.44+3.24+17.64+14.44+0.04=36.8
$$

因为协方差矩阵是对称的，所以直接补全：

所以：

$$
\tilde{X}_k^\top \tilde{X}_k = \begin{pmatrix}
10 & 15 & 19 \\
15 & 22.8 & 28.8 \\
19 & 28.8 & 36.8 \\
\end{pmatrix}
$$

除以4（因为$n-1=4$）得到协方差矩阵：

$$
\Sigma_k = \begin{pmatrix}
2.5 & 3.75 & 4.75 \\
3.75 & 5.7 & 7.2 \\
4.75 & 7.2 & 9.2 \\
\end{pmatrix}
$$



#### 第三步：求第一主成分

现在要求 $\Sigma_k$ 的最大特征值 $\lambda_1$ 和对应特征向量 $v_1$。

**略去繁琐细节**（用手算太爆炸了，一般用数值软件 eigendecomposition），但结果是：

（用Python或者计算器可以快速算）

大概可以得到最大特征向量 $v_1$（归一化后）形状像：

$$
v_1 \approx \begin{pmatrix}
0.39 \\
0.59 \\
0.71 \\
\end{pmatrix}
$$



#### 第四步：计算模块特征向量 $\text{ME}_k$

直接左乘：

$$
\text{ME}_k = \tilde{X}_k v_1
$$

逐行内积，得到5个样本的 $\text{ME}_k$ 数值，比如：

- 样本1：

$$
(-1)(0.39) + (-1.2)(0.59) + (-1.2)(0.71) = -0.39 - 0.708 - 0.852 \approx -1.95
$$

（可以继续算每一行，得到完整 $\text{ME}_k$）

### 🎯 为什么 PCA 要去中心化？（长一点的解释）

#### PCA 的目标

主成分分析（PCA）希望找出一个方向 \( v \)，使得数据在该方向上投影后的方差最大：

$$
v_1 = \arg\max_{\|v\| = 1} \operatorname{Var}(Xv)
$$

#### 那么，方差是怎么定义的？

标准样本方差定义为：

$$
\operatorname{Var}(x) = \frac{1}{n-1} \sum_{i=1}^n (x_i - \bar{x})^2
$$

注意：这里必须减去均值 \( \bar{x} \)，即需要先进行**中心化（centering）**。


### 如果不去中心化会怎样？

直接使用原始数据计算：

$$
X^\top X
$$

那么协方差中就混入了**均值的信息**。比如，所有样本整体偏移到右上角，会导致协方差增大，但这其实和**变量之间的真实关系**无关。

所以，**不去中心化**的 PCA，可能会错误地找到朝着“数据整体偏移方向”的主成分，而不是“变化最大”的方向。

#### 正确做法 ✅

首先对数据进行去中心化（每一列减去它的均值）：

$$
\tilde{X} = X - \bar{X}
$$

然后再计算协方差矩阵：

$$
\Sigma = \frac{1}{n-1} \tilde{X}^\top \tilde{X}
$$

最终在协方差矩阵上做特征值分解，得到的方向 \( v_1 \) 是真正的最大方差方向。



### 为什么要找第一主成分 $\(v_1\)$？


#### 1. **最初的目的是什么？**

回到最初的动机：

- 模块 \(M_k\) 有几十上百个基因。
- 每个样本有一堆基因表达值。
- 如果你直接拿这堆基因去做后续分析（比如性状关联），**又高维又冗余又噪声超大**，非常难搞。
  
我们想做的事情是：

✨ **用一个简单的数，尽可能总结这个模块的变化！**


#### 2. **为什么是第一主成分？**

因为第一主成分有非常厉害的数学特性：

| 特性 | 说明 |
|:--|:--|
| 最大方差 | 第一主成分方向，让模块内部样本投影后的方差最大。 |
| 信息保留最多 | 在所有可能的1维总结方法中，保留最多的信息量（最能区分样本）。 |
| 去除噪声 | 聚焦在主变化方向，自动忽略次要杂乱波动。 |

换句话说：

- 第一主成分 = **最大可能地保留模块表达变化的主趋势**！
- 其他任何"随便取个方向"的方法，信息量都不会比第一主成分多。


#### 3. **如果不用第一主成分，会怎样？**

- 用随机方向：得到的数据变化很小，分不清样本。
- 用某个单个基因代表：极容易被噪声、异常值搞崩。
- 简单平均所有基因：可能掩盖了真正的主导趋势。

第一主成分是**唯一经过数学严格证明**：
> **在保持数据尽可能分散（信息量最大）的同时，把高维模块压缩成一维表示**  
的最优方向。


#### PCA第一主成分的两个严肃数学性质

- 方差解释率最高（Explained variance maximized）
- 重建误差最小（Minimal reconstruction error in low dimension）

---

## 7. 模块与表型关联（Module-Trait Association）

设表型矩阵：

$$
Y \in \mathbb{R}^{n \times q}
$$

其中：
- \( n \)：样本数
- \( q \)：表型变量数（traits）
- 第 \( j \) 个表型变量记为 \( Y_{\cdot j} \in \mathbb{R}^n \)

对于每个模块 \( k \)，记其模块特征向量（Module Eigengene）为：

$$
\text{ME}_k \in \mathbb{R}^n
$$

则定义模块 \( k \) 与 trait \( j \) 的相关系数为：

$$
r_{kj} = \text{cor}(\text{ME}_k, Y_{\cdot j})
$$

即皮尔逊相关系数。



### 假设检验（Significance Testing）

为判断相关性是否显著，进行如下假设检验：

- 零假设：模块与 trait 无关，\( H_0: r_{kj} = 0 \)
- 备择假设：模块与 trait 存在线性相关性，\( H_1: r_{kj} \neq 0 \)

根据皮尔逊相关系数，可计算对应的 t 统计量：

$$
t = \frac{r_{kj} \cdot \sqrt{n - 2}}{\sqrt{1 - r_{kj}^2}}
$$

其自由度为：

$$
\text{df} = n - 2
$$

由此可计算对应的 p 值。



### 多重检验校正（Multiple Testing Correction）

由于存在多个模块与多个表型变量组合，总共会进行 \( K \times q \) 次检验，因此需要控制假阳性率。

常用方法是控制 FDR（False Discovery Rate），例如使用：

- Benjamini–Hochberg 方法（FDR 控制）
- Bonferroni 校正（更保守）

得到校正后的 p 值矩阵后，可进一步筛选具有统计显著性的模块–trait 对应关系。


---

# 总结流程

1. 输入表达矩阵 $X$
2. 计算基因相关矩阵 $S$
3. Soft-threshold 构建邻接矩阵 $A$
4. 计算拓扑重叠矩阵 $T$
5. 基于 $1-T$ 聚类识别模块 $M_k$
6. 提取模块特征向量 $\text{ME}_k$
7. 关联模块与表型 $Y$

---
